In [16]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
import pandas as pd

# read dataframe from file
cells_df = pd.read_csv('../eigenspike/pca_ephys_features.csv')
cells_df = cells_df.set_index('id')
cells_df.head()

,mouse_line,region,layer,hemisphere,dendrite_type,apical,has_morphology,has_reconstruction,normalized_depth,reference_space_id,...,ephys_ss_01,ephys_ss_02,ephys_ss_03,ephys_ss_04,ephys_ss_05,ephys_ss_06,ephys_ss_07,ephys_ss_08,ephys_ss_09,ephys_cluster
id,,,,,,,,,,,,,,,,,,,,,
324257146,Sst-IRES-Cre,VISp,4,left,aspiny,NaN,True,True,0.410398,9,...,1.170828,-1.163381,-1.093751,-0.253957,2.135071,-0.337337,1.784947,-1.081003,0.304944,3
469622566,Scnn1a-Tg3-Cre,VISp,5,right,spiny,truncated,True,False,0.423992,9,...,0.100590,0.391349,0.187695,0.196732,-0.570378,-0.041735,-0.291465,0.387696,-0.775480,25
328876201,Sst-IRES-Cre,VISp,5,left,aspiny,NaN,False,False,0.510874,9,...,-2.206795,2.194863,-1.924652,-1.294871,0.688790,0.695502,-0.851446,-1.023581,0.794630,4
466431949,Scnn1a-Tg3-Cre,VISl,4,left,spiny,truncated,True,False,0.464470,9,...,-0.848878,1.475682,-1.710938,0.626604,-0.345677,-0.824317,-1.126075,0.900409,-0.220645,4
396903227,Scnn1a-Tg3-Cre,VISp,5,right,spiny,truncated,False,False,0.497223,9,...,-0.128818,0.896799,-2.069474,-0.439117,-0.464414,0.827571,-0.393268,0.266873,0.687433,4


In [18]:
# fetch transform from api and put into cells_df dataframe
import urllib
import json
import numpy as np

ncomp = 12

def fetch_transform_from_api(id) :
    url = "http://api.brain-map.org/api/v2/data/query.json?criteria=model::DataSet,rma::criteria,[specimen_id$eq%d],pipe::list[xds$eq'id'],model::Alignment3d,rma::criteria,[aligned_id$in$xds]" % id
    response = urllib.urlopen(url) 
    data = json.loads(response.read())['msg'][0]
    output = np.zeros(12,dtype=np.float)
    for i in range(ncomp) :
        output[i] = data['tvr_%02d' % i]
    return output

for i in range(ncomp) :
    cells_df['ccf_transform_%02d' % i] = np.nan
    
start_label = 'ccf_transform_%02d' % 0
end_label = 'ccf_transform_%02d' % (ncomp-1)

for idx, c in cells_df.iterrows() :
    if c['has_reconstruction'] :
        cells_df.ix[idx,start_label:end_label] = fetch_transform_from_api(idx)
                                                                       

In [19]:
cid = 468120757
cells_df.ix[cid,start_label:end_label].values

array([0.050448800000000002, 0.087615299999999993, -1.0028900000000001,
       0.016404200000000001, 0.86731999999999998, -0.025378600000000001,
       -0.83085500000000001, 0.031206899999999999, -0.077089199999999997,
       7854.2399999999998, 484.82400000000001, 8623.7199999999993], dtype=object)

In [20]:
# use the sdk cache/function to assess morphology

from allensdk.core.cell_types_cache import CellTypesCache

# !! update this path so that it points to your external hard drive !!
ctc = CellTypesCache(manifest_file='D:/cell_types/manifest.json')

In [21]:
morphology = ctc.get_reconstruction(cid)

# make a compartment dictionary
compartment_dict = {}
for n in morphology.compartment_list :
    compartment_dict[n['id']] = n

In [41]:
import nrrd
# load 3D volumes
fname = 'D:\connectivity\cortical_coordinates\laplacian_10.nrrd'
laplacian, meta = nrrd.read(fname)

fname = 'D:\connectivity\cortical_coordinates\isocortex_mask_10.nrrd'
isocortex_mask, meta = nrrd.read(fname)

In [49]:
import create_morphology_volumes as cmv
import os

output_directory = './morphology'
for idx, c in cells_df.iterrows() :
    if c['has_reconstruction']:
        print idx
        exp_directory = os.path.join( output_directory, 'morphology_%d' % idx )
        if not os.path.exists(exp_directory):
            os.makedirs(exp_directory)
            
        morphology = ctc.get_reconstruction(idx)
        parameters = cells_df.ix[idx,start_label:end_label].values
        cmv.create_morphology_volumes( morphology.compartment_list, 
                                       parameters, exp_directory, 
                                      laplacian, isocortex_mask) 


324257146
469798159
348592897
464188580
324493977
467703703
469753383
320654829
314900022
320466871
320668879
313861608
320207387
336676216
323452245
471085676
321906005
466245544
319070795
314822529
471358165
466664172
324025371
321707905
318331342
370351753
469992918
318808427
471141261
464198958
397353539
313862373
471077857
314642645
323834998
322197160
471819401
313862306
396968721
313862167
464326095
314831019
382982932
471088062
471789504
333604946
466928405
469704261
464212183
467782778
321923685
354833767
354190013
318733871
471087830
330080937
468120757
325941643
324466858
386049446
314804042
323535089
327962063
395830185
323475862
469801569
397351623
471410185
471758398
471077468
333785935
327706437
313862022
